# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../starter_code/cities.csv") 
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,vaini,25,TO,1579991702,87,-21.20,-175.20,78.55,19.64
1,punta arenas,9,PH,1579991811,85,8.99,125.34,77.97,4.94
2,cruzilia,5,BR,1579991811,92,-21.84,-44.81,63.36,3.65
3,mahibadhoo,100,MV,1579991812,79,3.78,72.97,80.82,13.89
4,mys shmidta,100,RU,1579991812,96,62.06,48.62,4.10,3.89
...,...,...,...,...,...,...,...,...,...
546,bac lieu,59,RS,1579991865,71,45.39,19.24,37.99,5.61
547,ilinskiy,80,RU,1579991865,62,55.62,38.12,33.01,7.00
548,kumbo,72,CM,1579991866,23,6.20,10.67,60.67,3.49
549,nyuksenitsa,98,RU,1579991866,92,60.41,44.24,12.16,9.01


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]
locations = locations.dropna()   

# Store Humidity in humidity
humidity = weather_data["Humidity"]
humidity = humidity.dropna()


In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] > 70) & (weather_data["Max Temp"] < 80)].dropna()

narrowed_city_df


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
119,umkomaas,0,ZA,1579991822,89,-30.21,30.80,75.99,1.01
145,new norfolk,0,LR,1579991825,95,6.65,-10.59,74.14,0.78
353,remedios,0,CU,1579991846,70,22.49,-79.55,71.92,7.99
464,providencia,0,MX,1579991749,43,28.72,-111.58,79.00,4.00
528,buchanan,0,LR,1579991864,94,5.88,-10.05,76.21,0.67


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
119,umkomaas,ZA,-30.21,30.80,
145,new norfolk,LR,6.65,-10.59,
353,remedios,CU,22.49,-79.55,
464,providencia,MX,28.72,-111.58,
528,buchanan,LR,5.88,-10.05,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [8]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
   # time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 119: umkomaas.
Closest hotel in umkomaas is Alien Hotel.
------------
Retrieving Results for Index 145: new norfolk.
Missing field/result... skipping.
------------
Retrieving Results for Index 353: remedios.
Closest hotel in remedios is Hotel E Real.
------------
Retrieving Results for Index 464: providencia.
Missing field/result... skipping.
------------
Retrieving Results for Index 528: buchanan.
Closest hotel in buchanan is Sparks Hotel.
------------
-------End of Search-------


In [10]:

hotel_df = hotel_df.rename(columns = {'City': 'Cities'})
hotel_df


,Cities,Country,Lat,Lng,Hotel Name
119,umkomaas,ZA,-30.21,30.80,Alien Hotel
145,new norfolk,LR,6.65,-10.59,
353,remedios,CU,22.49,-79.55,Hotel E Real
464,providencia,MX,28.72,-111.58,
528,buchanan,LR,5.88,-10.05,Sparks Hotel


In [12]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>Cities</dt><dd>{Cities}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:

# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig



Figure(layout=FigureLayout(height='420px'))